# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2c1bc38aaa-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Players explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sport. Compare players per countries, awards of the players and participation in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football player

2. Identify the BGP for association football club

3. Identify the BGP for association football league

4. Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

5. Identify the BGP for the ballon d'Or award

6. Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

7. Identify the BGP of the FIFA World Cup

8. Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

9. Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).


## Task 1
Identify the BGP for association football player

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [3]:
# CR7 properties 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [4]:
# query 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
1


In [5]:
# query 
queryString = """
SELECT DISTINCT ?name
WHERE {
   # bind something
   ?s wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
ORDER BY DESC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', '色')]
[('name', '梅林')]
[('name', '外郭団体')]
[('name', 'ㅇ')]
[('name', 'ひ')]
[('name', 'त')]
[('name', 'ट')]
[('name', 'ञ')]
[('name', 'झ')]
[('name', 'ज')]
[('name', 'छ')]
[('name', 'च')]
[('name', 'ग')]
[('name', 'چهارسوق')]
[('name', 'فلسطين.')]
[('name', 'сubic surface')]
[('name', 'сollegium')]
[('name', 'сollection of fairy tales')]
[('name', 'сhurch college')]
[('name', 'сhorale setting')]
[('name', "сhildren's art school")]
[('name', 'Сosmism')]
[('name', 'ООО')]
[('name', 'μ-recursive function')]
[('name', 'δ-carotene')]
[('name', 'γ-lactone')]
[('name', 'γ-himachalene')]
[('name', 'β-phellandrene')]
[('name', 'β-lactam antibiotic')]
[('name', 'β-lactam')]
[('name', 'β-ionone')]
[('name', 'β-himachalene')]
[('name', 'β-elemene')]
[('name', 'β-carene')]
[('name', 'β-bromo-β-nitrostyrene')]
[('name', 'α/β-lactose')]
[('name', 'α-phellandrene')]
[('name', 'α-himachalene')]
[('name', 'ținut')]
[('name', 'Župa')]
[('name', 'Žudr')]
[('name', 'Žalý')]
[('name', 'šumpe

In [6]:
# query 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [7]:
# query 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P641 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
1


In [8]:
#all soccer players
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   ?player wdt:P641 wd:Q2736.
   # get the label
   ?player sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('player', 'http://www.wikidata.org/entity/Q1024094'), ('name', "2010 CONCACAF Women's Gold Cup")]
[('player', 'http://www.wikidata.org/entity/Q10271305'), ('name', '2014 FIFA World Cup venues')]
[('player', 'http://www.wikidata.org/entity/Q1064712'), ('name', '1970 FIFA World Cup qualification')]
[('player', 'http://www.wikidata.org/entity/Q1082132'), ('name', '2004 UEFA European Under-17 Football Championship')]
[('player', 'http://www.wikidata.org/entity/Q11336831'), ('name', 'list of Vegalta Sendai players')]
[('player', 'http://www.wikidata.org/entity/Q1137516'), ('name', '2009–2010 Coupe de France')]
[('player', 'http://www.wikidata.org/entity/Q1142956'), ('name', "2011 Women's Professional Soccer season")]
[('player', 'http://www.wikidata.org/entity/Q11782976'), ('name', '1994–95 in Ukrainian football')]
[('player', 'http://www.wikidata.org/entity/Q1191053'), ('name', "2009–10 UEFA Women's Champions League")]
[('player', 'http://www.wikidata.org/entity/Q1279521'), ('nam

In [9]:
#all soccer players
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    # bind something
   wd:Q11571 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
1


In [10]:
#all soccer people, not players
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   ?player wdt:P641 wd:Q2736;
           wdt:P31  wd:Q5.
   # get the label
   ?player sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q259758'), ('name', 'Andrés Mendoza')]
[('player', 'http://www.wikidata.org/entity/Q8135287'), ('name', 'Naoki Kurimoto')]
[('player', 'http://www.wikidata.org/entity/Q8157342'), ('name', 'Lamberto Anichini')]
[('player', 'http://www.wikidata.org/entity/Q8196829'), ('name', 'Alvin Avinesh')]
[('player', 'http://www.wikidata.org/entity/Q8199060'), ('name', 'André Advíncula')]
[('player', 'http://www.wikidata.org/entity/Q8199367'), ('name', 'Andrés Mendoza')]
[('player', 'http://www.wikidata.org/entity/Q8201109'), ('name', 'Antonino Gart')]
[('player', 'http://www.wikidata.org/entity/Q8204020'), ('name', 'Argemiro Veiga Gonçalves')]
[('player', 'http://www.wikidata.org/entity/Q8238626'), ('name', 'Jiří Adamec')]
[('player', 'http://www.wikidata.org/entity/Q29162'), ('name', 'Tomáš Rosický')]
[('player', 'http://www.wikidata.org/entity/Q3027395'), ('name', 'Diego Héctor Garay')]
[('player', 'http://www.wikidata.org/entity/Q8351764'), ('n

In [11]:
#all soccer player 
queryString = """
SELECT DISTINCT ?player ?name
WHERE {
   # bind something
   ?player wdt:P106 wd:Q937857;           
           sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q259758'), ('name', 'Andrés Mendoza')]
[('player', 'http://www.wikidata.org/entity/Q8135287'), ('name', 'Naoki Kurimoto')]
[('player', 'http://www.wikidata.org/entity/Q8157342'), ('name', 'Lamberto Anichini')]
[('player', 'http://www.wikidata.org/entity/Q8196829'), ('name', 'Alvin Avinesh')]
[('player', 'http://www.wikidata.org/entity/Q8199060'), ('name', 'André Advíncula')]
[('player', 'http://www.wikidata.org/entity/Q8199367'), ('name', 'Andrés Mendoza')]
[('player', 'http://www.wikidata.org/entity/Q8201109'), ('name', 'Antonino Gart')]
[('player', 'http://www.wikidata.org/entity/Q8204020'), ('name', 'Argemiro Veiga Gonçalves')]
[('player', 'http://www.wikidata.org/entity/Q8238626'), ('name', 'Jiří Adamec')]
[('player', 'http://www.wikidata.org/entity/Q29162'), ('name', 'Tomáš Rosický')]
[('player', 'http://www.wikidata.org/entity/Q3027395'), ('name', 'Diego Héctor Garay')]
[('player', 'http://www.wikidata.org/entity/Q8351764'), ('n

Final query for this task

In [12]:
#BGP for association football player
queryString = """
SELECT *
WHERE {
    
    BIND(wd:Q937857 AS ?iri)
    ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
1


## Task 2
Identify the BGP for association football club

In [13]:
#CR7 Properties
queryString = """
SELECT DISTINCT ?p ?name ?o
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?o.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('o', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('o', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league'), ('o', 'Liga Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'given name'), ('o', 'Cristiano')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('o', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'), ('o', 'Portugal Olympic football team')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('o', 'UEFA Euro 2020')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('name', 'family name'), ('o', 'dos Santos')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('o', 'UEFA Team of

In [14]:
#all soccer teams (nationals included)
queryString = """
SELECT DISTINCT ?team ?name
WHERE {
   # soccer players
   ?player wdt:P106 wd:Q937857.
   # retrieve clubs
   ?player wdt:P54 ?team.
   ?team sc:name ?name.        
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q751816'), ('name', 'Vatican City national association football team')]
[('team', 'http://www.wikidata.org/entity/Q14252413'), ('name', 'Rapid JC')]
[('team', 'http://www.wikidata.org/entity/Q14919844'), ('name', "Fortuna '54")]
[('team', 'http://www.wikidata.org/entity/Q3375656'), ('name', 'Persekabpas Pasuruan')]
[('team', 'http://www.wikidata.org/entity/Q5903612'), ('name', 'Horizonte Futebol Clube')]
[('team', 'http://www.wikidata.org/entity/Q16842749'), ('name', 'Klippans FF')]
[('team', 'http://www.wikidata.org/entity/Q16844147'), ('name', 'Toronto Lady Lynx')]
[('team', 'http://www.wikidata.org/entity/Q28444666'), ('name', "West Virginia Mountaineers women's soccer")]
[('team', 'http://www.wikidata.org/entity/Q3626845'), ('name', 'A.S.D. Anziolavinio')]
[('team', 'http://www.wikidata.org/entity/Q3860587'), ('name', 'Moncalieri Calcio')]
[('team', 'http://www.wikidata.org/entity/Q4774840'), ('name', 'Antigua and Barbuda national u

In [15]:
#vatican team 
queryString = """
SELECT DISTINCT ?p ?name ?oN
WHERE {
   # bind something
   wd:Q751816 ?p ?o.          
   ?p sc:name ?name.
   ?o sc:name ?oN
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('oN', 'Vatican City')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('oN', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('oN', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('oN', 'national association football team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('oN', 'Category:Vatican City national association football team')]
5


In [16]:
#hopefully not national team
queryString = """
SELECT DISTINCT ?p ?name ?o ?oN
WHERE {
   # bind something
   wd:Q867921 ?p ?o.          
   ?p sc:name ?name.
   ?o sc:name ?oN
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after'), ('o', 'http://www.wikidata.org/entity/Q848287'), ('oN', 'Livingston')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q848287'), ('oN', 'Livingston')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oN', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oN', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q476028'), ('oN', 'association football club')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league'), ('o', 'http://www.wikidata.org/entity/Q14377162'), ('oN', 'Scottish Premiership')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in th

In [17]:
#all soccer clubs maybe
queryString = """
SELECT DISTINCT ?team ?name
WHERE {   
   ?player wdt:P106 wd:Q937857; # soccer players   
           wdt:P54 ?team.
   ?team wdt:P31 wd:Q476028; #only clubs
           sc:name ?name.        
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q4647107'), ('name', 'A.D. Dili Leste')]
[('team', 'http://www.wikidata.org/entity/Q8186475'), ('name', 'Tahuichi Academy')]
[('team', 'http://www.wikidata.org/entity/Q2631783'), ('name', 'Cizrespor')]
[('team', 'http://www.wikidata.org/entity/Q3448775'), ('name', 'FK Sloboda Mrkonjić Grad')]
[('team', 'http://www.wikidata.org/entity/Q5100109'), ('name', 'Chinandega FC')]
[('team', 'http://www.wikidata.org/entity/Q7300846'), ('name', 'Real Arroyo Seco')]
[('team', 'http://www.wikidata.org/entity/Q14252413'), ('name', 'Rapid JC')]
[('team', 'http://www.wikidata.org/entity/Q14919844'), ('name', "Fortuna '54")]
[('team', 'http://www.wikidata.org/entity/Q3375656'), ('name', 'Persekabpas Pasuruan')]
[('team', 'http://www.wikidata.org/entity/Q6414993'), ('name', 'Southwick F.C.')]
[('team', 'http://www.wikidata.org/entity/Q16842749'), ('name', 'Klippans FF')]
[('team', 'http://www.wikidata.org/entity/Q3626845'), ('name', 'A.S.D. Anziolavinio'

In [18]:
#all soccer clubs
queryString = """
SELECT DISTINCT ?p ?name
WHERE {   
   wd:Q476028 ?p ?o.
   ?p sc:name ?name.        
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

Final query for this task

In [19]:
#BGP for association football club. Idk, anyway found the node
queryString = """
SELECT *
WHERE {
    
    BIND(wd:Q476028 AS ?iri)
    ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
1


## Task 3
Identify the BGP for association football league

In [20]:
#props of a league
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {   
   wd:Q14377162 ?p ?o. #scottish league
   ?p sc:name ?pName.   
   ?o sc:name ?oName.        

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q122'), ('oName', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P2541'), ('pName', 'operating area'), ('o', 'http://www.wikidata.org/entity/Q22'), ('oName', 'Scotland')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q31930761'), ('oName', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3270632'), ('oName', 'national championship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), 

In [21]:
#all leagues maybe
queryString = """
SELECT DISTINCT ?league ?name
WHERE {   
    ?league wdt:P31 wd:Q623109;
            wdt:P641 wd:Q2736;
            sc:name ?name.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q4688766'), ('name', 'Aetolian League')]
[('league', 'http://www.wikidata.org/entity/Q15141004'), ('name', 'Nationalliga A')]
[('league', 'http://www.wikidata.org/entity/Q12408669'), ('name', 'Israeli Noar Premier League')]
[('league', 'http://www.wikidata.org/entity/Q19871058'), ('name', 'Great Lakes Premier League')]
[('league', 'http://www.wikidata.org/entity/Q22032278'), ('name', 'Dublin Womens Soccer League')]
[('league', 'http://www.wikidata.org/entity/Q27489767'), ('name', "Colombian Women's Football League")]
[('league', 'http://www.wikidata.org/entity/Q483045'), ('name', 'Korea National League')]
[('league', 'http://www.wikidata.org/entity/Q1207912'), ('name', 'China League One')]
[('league', 'http://www.wikidata.org/entity/Q2289847'), ('name', 'LigaPro Ecuador')]
[('league', 'http://www.wikidata.org/entity/Q252836'), ('name', 'Coppa Italia Lega Pro')]
[('league', 'http://www.wikidata.org/entity/Q25762'), ('name', '1. Liga Pr

In [22]:
#props of Serie A
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {   
   wd:Q194052 ?p ?o. #scottish league
   ?p sc:name ?pName.   
   ?o sc:name ?oName.        

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q122'), ('oName', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('pName', 'league level above'), ('o', 'http://www.wikidata.org/entity/Q15804'), ('oName', 'Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('pName', 'league level below'), ('o', 'http://www.wikidata.org/entity/Q607965'), ('oName', 'Serie C')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q31930761'), ('oName', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', '

In [23]:
#props of true Serie A
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {   
   wd:Q15804 ?p ?o. 
   ?p sc:name ?pName.   
   ?o sc:name ?oName.        

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('pName', 'league level below'), ('o', 'http://www.wikidata.org/entity/Q194052'), ('oName', 'Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6807104'), ('oName', 'Category:Serie A')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q122'), ('oName', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q631'), ('oName', 'FC Internazionale Milano')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subc

In [24]:
#all leagues 
queryString = """
SELECT DISTINCT ?league ?name
WHERE {   
    ?league wdt:P31 wd:Q623109; #instance of sports league
            wdt:P641 wd:Q2736; #sport soccer
            sc:name ?name.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q4688766'), ('name', 'Aetolian League')]
[('league', 'http://www.wikidata.org/entity/Q15141004'), ('name', 'Nationalliga A')]
[('league', 'http://www.wikidata.org/entity/Q12408669'), ('name', 'Israeli Noar Premier League')]
[('league', 'http://www.wikidata.org/entity/Q19871058'), ('name', 'Great Lakes Premier League')]
[('league', 'http://www.wikidata.org/entity/Q22032278'), ('name', 'Dublin Womens Soccer League')]
[('league', 'http://www.wikidata.org/entity/Q27489767'), ('name', "Colombian Women's Football League")]
[('league', 'http://www.wikidata.org/entity/Q483045'), ('name', 'Korea National League')]
[('league', 'http://www.wikidata.org/entity/Q1207912'), ('name', 'China League One')]
[('league', 'http://www.wikidata.org/entity/Q2289847'), ('name', 'LigaPro Ecuador')]
[('league', 'http://www.wikidata.org/entity/Q252836'), ('name', 'Coppa Italia Lega Pro')]
[('league', 'http://www.wikidata.org/entity/Q25762'), ('name', '1. Liga Pr

In [25]:
#all association football leagues 
queryString = """
SELECT DISTINCT ?league ?name
WHERE {   
    ?league wdt:P31 wd:Q15991303; #instance of association football league            
            sc:name ?name.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q304228'), ('name', 'Uzbekistan Pro League')]
[('league', 'http://www.wikidata.org/entity/Q2950655'), ('name', 'Ligue 2')]
[('league', 'http://www.wikidata.org/entity/Q2951924'), ('name', "Moldovan women's football championship")]
[('league', 'http://www.wikidata.org/entity/Q973312'), ('name', 'Championnat National 3')]
[('league', 'http://www.wikidata.org/entity/Q2792041'), ('name', '3. divisjon')]
[('league', 'http://www.wikidata.org/entity/Q1162407'), ('name', 'Nemzeti Bajnokság III')]
[('league', 'http://www.wikidata.org/entity/Q1476473'), ('name', 'Landesliga Niederrhein')]
[('league', 'http://www.wikidata.org/entity/Q162533'), ('name', 'Russian Football National League Competition')]
[('league', 'http://www.wikidata.org/entity/Q1707697'), ('name', 'Liga III')]
[('league', 'http://www.wikidata.org/entity/Q219586'), ('name', 'A-League')]
[('league', 'http://www.wikidata.org/entity/Q238113'), ('name', 'Slovak Super Liga')]
[('leagu

Final query for this task

In [26]:
#BGP for association football league. Idk, anyway found the node
queryString = """
SELECT *
WHERE {
    
    BIND(wd:Q15991303 AS ?iri)
    ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
1


## Task 4
Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

In [27]:
#all props of Italy
queryString = """
SELECT DISTINCT ?p ?pName 
WHERE {   
    wd:Q38 ?p ?o.
    ?p sc:name ?pName.
    ?o sc:name ?oName.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pName', 'flag')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pName', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('pName', 'category of associated people')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pName', 'diplomatic relation')]
[('p', 'http://www.wikidata.org/prop/direct/P2852'), ('pName', 'emergency telephone number')]
[('p', 'http://www.wikidata.org/prop/direct/P2853'), ('pName', 'electrical plug type')]
[('p', 'http://www.wikidata.org/prop/direct/P5658'), ('pName', 'railway traffic side')]
[('p', 'http://www.wikidata.org/prop/direct/P206'), ('pName', 'located in or next to body of water')]
[('p', 'http://w

In [28]:
#country of Italy
queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {   
    wd:Q38 wdt:P17 ?o.
    ?o sc:name ?oName.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
1


In [29]:
#all continents
queryString = """
SELECT DISTINCT ?o ?name
WHERE {   
    ?s wdt:P30 ?o.
    ?o sc:name ?name.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q60140'), ('name', 'Indian subcontinent')]
[('o', 'http://www.wikidata.org/entity/Q55643'), ('name', 'Oceania')]
[('o', 'http://www.wikidata.org/entity/Q19464773'), ('name', 'Northern America and Mexico')]
[('o', 'http://www.wikidata.org/entity/Q1239'), ('name', 'Indian Ocean')]
[('o', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('o', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('o', 'http://www.wikidata.org/entity/Q4412'), ('name', 'West Africa')]
[('o', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('o', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('o', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('o', 'http://www.wikidata.org/entity/Q51'), ('name', 'Antarctica')]
[('o', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('o', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('o', 'http://www.wikidata.org/entity/Q620

In [30]:
#Italy instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {   
    wd:Q38 wdt:P31 ?o.
    ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
2


In [31]:
#european countries
queryString = """
SELECT DISTINCT ?country ?name
WHERE {   
    ?country wdt:P30 wd:Q46;   #continent Europe
             wdt:P31 wd:Q6256; #instance of country
             sc:name ?name.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q756617'), ('name', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q907112'), ('name', 'Transnistria')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('country', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[(

In [32]:
#italy leagues
queryString = """
SELECT  ?league ?name 
WHERE {   
    
    ?league wdt:P31 wd:Q15991303; #instance of association football league
            wdt:P17 wd:Q38.
    OPTIONAL{?league sc:name ?name.}
}
GROUP BY ?country ?name
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q830717'), ('name', 'Lega Pro Seconda Divisione')]
[('league', 'http://www.wikidata.org/entity/Q1063016'), ('name', 'Prima Categoria')]
[('league', 'http://www.wikidata.org/entity/Q645148'), ('name', 'Eccellenza')]
[('league', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
[('league', 'http://www.wikidata.org/entity/Q102016095')]
[('league', 'http://www.wikidata.org/entity/Q607965'), ('name', 'Serie C')]
[('league', 'http://www.wikidata.org/entity/Q677397'), ('name', 'Lega Pro Prima Divisione')]
[('league', 'http://www.wikidata.org/entity/Q3718140'), ('name', 'Eccellenza Marche')]
[('league', 'http://www.wikidata.org/entity/Q3229032'), ('name', 'LNPB')]
[('league', 'http://www.wikidata.org/entity/Q3718114'), ('name', 'Eccellenza Lombardy')]
[('league', 'http://www.wikidata.org/entity/Q3718185'), ('name', 'Eccellenza Piedmont and Aosta Valley')]
[('league', 'http://www.wikidata.org/entity/Q86506910'), ('name', 'eSerie A

In [33]:
#nLeagues test
queryString = """
SELECT  ?country ?name (COUNT( ?league) AS ?Nleagues)
WHERE {   
    ?country wdt:P30 wd:Q46;   #continent Europe
             wdt:P31 wd:Q6256; #instance of country
             sc:name ?name.
    ?league wdt:P31 wd:Q15991303; #instance of association football league
            wdt:P17 ?country.
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania'), ('Nleagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra'), ('Nleagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('Nleagues', '12')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus'), ('Nleagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('Nleagues', '4')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('Nleagues', '5')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus'), ('Nleagues', '13')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('Nleagues', '199')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('Nleagues', '8')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('Nleagues', '14')]
[('country', 'http://www.wikidata.org/entity/Q142'), 

Final query for this task

In [34]:
#number of association football leagues per european country
queryString = """
SELECT  ?country ?name (COUNT( ?league) AS ?Nleagues)
WHERE {   
    ?country wdt:P30 wd:Q46;   #continent Europe
             wdt:P31 wd:Q6256; #instance of country
             sc:name ?name.
    ?league wdt:P31 wd:Q15991303; #instance of association football league
            wdt:P17 ?country.
}
GROUP BY ?country ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania'), ('Nleagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra'), ('Nleagues', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('Nleagues', '12')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus'), ('Nleagues', '3')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('Nleagues', '4')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('Nleagues', '5')]
[('country', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus'), ('Nleagues', '13')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('Nleagues', '199')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('Nleagues', '8')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('Nleagues', '14')]
[('country', 'http://www.wikidata.org/entity/Q142'), 

## Task 5
Identify the BGP for the ballon d'Or award

In [35]:
#CR7 awards
queryString = """
SELECT  ?o ?name
WHERE {   
    wd:Q11571 wdt:P166 ?o.
    ?o sc:name ?name.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('o', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('o', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('o', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('o', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('o', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEFA Men's Player of the Year Award")]
[('o', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
[('o', 'http://www.wikidata.org/entity/Q20860117'), ('name', 'Officer of the Order of Prince Henry')]
[('o', 'http://www.wikidata.org/entity/Q20859673'), ('name', 'Grand Officer of the Order of Prince Henry')]
[('o', 'http://www.wikidata.org/entity/Q2986505'), ('name', 'Sir Matt Busby Player of the Year')]
[('o', 'http://www.wikidata.org/entit

In [36]:
#ballon d'Or all props
queryString = """
SELECT DISTINCT ?p ?name
WHERE {   
    wd:Q166177 ?p ?o.
    ?p sc:name ?name.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2572'), ('name', 'hashtag')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3222'), ('name', 'NE.se ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P64

In [37]:
#ballon d'Or winners
queryString = """
SELECT DISTINCT ?o ?name
WHERE {   
    wd:Q166177 wdt:P1346 ?o.
    ?o sc:name ?name.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q162043'), ('name', 'Eusébio')]
[('o', 'http://www.wikidata.org/entity/Q128829'), ('name', 'Michael Owen')]
[('o', 'http://www.wikidata.org/entity/Q102027'), ('name', 'Fabio Cannavaro')]
[('o', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus')]
[('o', 'http://www.wikidata.org/entity/Q178628'), ('name', 'Paolo Rossi')]
[('o', 'http://www.wikidata.org/entity/Q216140'), ('name', 'Stanley Matthews')]
[('o', 'http://www.wikidata.org/entity/Q483145'), ('name', 'Luís Figo')]
[('o', 'http://www.wikidata.org/entity/Q483837'), ('name', 'Luka Modrić')]
[('o', 'http://www.wikidata.org/entity/Q39444'), ('name', 'Ronaldinho')]
[('o', 'http://www.wikidata.org/entity/Q483629'), ('name', 'Marco van Basten')]
[('o', 'http://www.wikidata.org/entity/Q17163'), ('name', 'Johan Cruyff')]
[('o', 'http://www.wikidata.org/entity/Q220700'), ('name', 'Omar Sívori')]
[('o', 'http://www.wikidata.org/entity/Q4261'), ('name', 'Michel Platini')]
[('o',

Final query for this task

In [38]:
#BGP for ballon d'Or 
queryString = """
SELECT *
WHERE {
    
    BIND(wd:Q166177 AS ?iri)
    ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
1


## Task 6
Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

In [39]:
# CR7 properties 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [40]:
#countries and count 
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(DISTINCT ?player) AS ?Nwinners)
WHERE {
   ?player wdt:P106 wd:Q937857; #football players
           wdt:P27 ?country;    #citizenship
           wdt:P166 wd:Q166177. #won ballon d'or
   ?country sc:name ?countryName.  
}
GROUP BY ?country ?countryName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q219'), ('countryName', 'Bulgaria'), ('Nwinners', '1')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('Nwinners', '1')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('countryName', 'Ukraine'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia'), ('Nwinners', '1')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('Nwinners', '5')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('Nwinners

In [41]:
#countries w2 ballon d'or 
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(DISTINCT ?player) AS ?Nwinners)
WHERE {
   ?player wdt:P106 wd:Q937857; #football players
           wdt:P27 ?country;    #citizenship
           wdt:P166 wd:Q166177. #won ballon d'or
   ?country sc:name ?countryName.  
}
GROUP BY ?country ?countryName
HAVING (COUNT(DISTINCT ?player) > 1)
ORDER BY DESC(?Nwinners)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('Nwinners', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('Nwinners', '6')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('Nwinners', '5')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('countryName', 'Ukraine'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of the

Final query for this task

In [42]:
# write your final query
#countries w2 ballon d'or 
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(DISTINCT ?player) AS ?Nwinners)
WHERE {
   ?player wdt:P106 wd:Q937857; #football players
           wdt:P27 ?country;    #citizenship
           wdt:P166 wd:Q166177. #won ballon d'or
   ?country sc:name ?countryName.  
}
GROUP BY ?country ?countryName
HAVING (COUNT(DISTINCT ?player) > 1)
ORDER BY DESC(?Nwinners)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('Nwinners', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('Nwinners', '6')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('Nwinners', '5')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('Nwinners', '4')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('countryName', 'Ukraine'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union'), ('Nwinners', '3')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of the

## Task 7
Identify the BGP of the FIFA World Cup

In [43]:
# CR7 participant in
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q373501'), ('name', 'UEFA Euro 2020')]
[('o', 'http://www.wikidata.org/entity/Q102920'), ('name', 'UEFA Euro 2004')]
[('o', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q189571'), ('name', 'UEFA Euro 2016')]
[('o', 'http://www.wikidata.org/entity/Q22669'), ('name', 'UEFA Euro 2012')]
[('o', 'http://www.wikidata.org/entity/Q241864'), ('name', 'UEFA Euro 2008')]
[('o', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('o', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q79859'), ('name', '2014 FIFA World Cup')]
[('o', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
11


In [44]:
# 2018 FWC props
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('name', 'final event'), ('obj', 'http://www.wikidata.org/entity/Q31043671'), ('objName', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objName', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('obj', 'http://www.wikidata.org/entity/Q202163'), ('objName', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('objName', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q27020041'), ('objName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('

In [45]:
# all FWCs
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P3450 wd:Q19317. #sports season of league or competition = FIFA World Cup
   # get the label
   ?s sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q101751'), ('name', '1994 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q132664'), ('name', '1970 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q46934'), ('name', '1982 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q46938'), ('name', '1986 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q46943'), ('name', '1978 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q47735'), ('name', '2002 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q79859'), ('name', '2014 FIFA World Cup')]
[('

In [46]:
# all FWC props
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q19317 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct

Final query for this task

In [47]:
#BGP for FIFA World Cup
queryString = """
SELECT *
WHERE {
    
    BIND(wd:Q19317 AS ?iri)
    ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
1


## Task 8
Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

In [48]:
# prova 1, include anche non giocatori
queryString = """
SELECT DISTINCT ?p ?name (COUNT(?comp) AS ?Nparticipations)
WHERE {   
   ?p  sc:name ?name;
       wdt:P1344 ?comp.       #participant in comp
   ?comp wdt:P3450 wd:Q19317. #comp a FWC
}
GROUP BY ?p ?name
HAVING(COUNT(?comp) > 4)
ORDER BY DESC(?Nparticipations)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q83459'), ('name', 'Brazil national football team'), ('Nparticipations', '20')]
[('p', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team'), ('Nparticipations', '18')]
[('p', 'http://www.wikidata.org/entity/Q43310'), ('name', 'Germany national association football team'), ('Nparticipations', '18')]
[('p', 'http://www.wikidata.org/entity/Q79800'), ('name', 'Argentina national football team'), ('Nparticipations', '15')]
[('p', 'http://www.wikidata.org/entity/Q164089'), ('name', 'Mexico national football team'), ('Nparticipations', '15')]
[('p', 'http://www.wikidata.org/entity/Q47774'), ('name', 'France national association football team'), ('Nparticipations', '14')]
[('p', 'http://www.wikidata.org/entity/Q42267'), ('name', 'Spain national association football team'), ('Nparticipations', '14')]
[('p', 'http://www.wikidata.org/entity/Q47762'), ('name', 'England national association football team'), ('

In [49]:
# prova 2
queryString = """
SELECT DISTINCT ?p ?name (COUNT(?comp) AS ?Nparticipations)
WHERE {   
   ?p  sc:name ?name;
       wdt:P106 wd:Q937857;   #soccer player
       wdt:P1344 ?comp.       #participant in comp
   ?comp wdt:P3450 wd:Q19317. #comp a FWC
}
GROUP BY ?p ?name
HAVING(COUNT(?comp) > 4)
ORDER BY DESC(?Nparticipations)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus'), ('Nparticipations', '5')]
[('p', 'http://www.wikidata.org/entity/Q186330'), ('name', 'Rafael Márquez'), ('Nparticipations', '5')]
[('p', 'http://www.wikidata.org/entity/Q265428'), ('name', 'Antonio Carbajal'), ('Nparticipations', '5')]
[('p', 'http://www.wikidata.org/entity/Q68060'), ('name', 'Gianluigi Buffon'), ('Nparticipations', '5')]
4


Final query for this task

In [50]:
#players with more than 4 FWC participations
queryString = """
SELECT DISTINCT ?p ?name (COUNT(?comp) AS ?Nparticipations)
WHERE {   
   ?p  sc:name ?name;
       wdt:P106 wd:Q937857;   #soccer player
       wdt:P1344 ?comp.       #participant in comp
   ?comp wdt:P3450 wd:Q19317. #comp a FWC
}
GROUP BY ?p ?name
HAVING(COUNT(?comp) > 4)
ORDER BY DESC(?Nparticipations)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus'), ('Nparticipations', '5')]
[('p', 'http://www.wikidata.org/entity/Q186330'), ('name', 'Rafael Márquez'), ('Nparticipations', '5')]
[('p', 'http://www.wikidata.org/entity/Q265428'), ('name', 'Antonio Carbajal'), ('Nparticipations', '5')]
[('p', 'http://www.wikidata.org/entity/Q68060'), ('name', 'Gianluigi Buffon'), ('Nparticipations', '5')]
4


## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).

In [51]:
#countries that won at least twice
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?comp) AS ?Nwin)
WHERE {      
    ?comp wdt:P3450 wd:Q19317; #comp a FWC
          wdt:P1346 ?team.     #team is the winner
    ?team wdt:P17 ?country.    #team represents country
    ?country sc:name ?name.
}
GROUP BY ?country ?name
HAVING(COUNT(?comp) > 1)
ORDER BY DESC(?Nwin)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('Nwin', '5')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('Nwin', '4')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('Nwin', '4')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('Nwin', '2')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('Nwin', '2')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('name', 'Uruguay'), ('Nwin', '2')]
6


In [52]:
#players with ballon d'or and country that has won at least one world cup
queryString = """
SELECT DISTINCT ?player ?name
WHERE {      
    ?comp wdt:P3450 wd:Q19317;   #comp a FWC
          wdt:P1346 ?team.       #team is the winner
    ?team wdt:P17 ?country.      #team represents country
    ?player sc:name ?name;
            wdt:P106 wd:Q937857; #football players
            wdt:P27 ?country;    #citizenship
            wdt:P166 wd:Q166177. #won ballon d'or
        
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q123118'), ('name', 'Raymond Kopa')]
[('player', 'http://www.wikidata.org/entity/Q13494'), ('name', 'Jean-Pierre Papin')]
[('player', 'http://www.wikidata.org/entity/Q4261'), ('name', 'Michel Platini')]
[('player', 'http://www.wikidata.org/entity/Q1835'), ('name', 'Zinedine Zidane')]
[('player', 'http://www.wikidata.org/entity/Q171583'), ('name', 'Bobby Charlton')]
[('player', 'http://www.wikidata.org/entity/Q152832'), ('name', 'Kevin Keegan')]
[('player', 'http://www.wikidata.org/entity/Q164521'), ('name', 'George Best')]
[('player', 'http://www.wikidata.org/entity/Q128829'), ('name', 'Michael Owen')]
[('player', 'http://www.wikidata.org/entity/Q233565'), ('name', 'Denis Law')]
[('player', 'http://www.wikidata.org/entity/Q216140'), ('name', 'Stanley Matthews')]
[('player', 'http://www.wikidata.org/entity/Q529207'), ('name', 'Ronaldo')]
[('player', 'http://www.wikidata.org/entity/Q39444'), ('name', 'Ronaldinho')]
[('player', 'http://w

In [53]:
#prova per finale
queryString = """
SELECT DISTINCT ?player ?name
WHERE {          
    ?player sc:name ?name;
            wdt:P106 wd:Q937857; #football players
            wdt:P27 ?country;    #citizenship
            wdt:P166 wd:Q166177. #won ballon d'or
    {
    SELECT DISTINCT ?country (COUNT(?comp) AS ?Nwin) WHERE {      
        ?comp wdt:P3450 wd:Q19317; #comp a FWC
              wdt:P1346 ?team.     #team is the winner
        ?team wdt:P17 ?country.    #team represents country
    }
    GROUP BY ?country ?name
    HAVING(COUNT(?comp) > 1)
    LIMIT 100
    }
   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q4261'), ('name', 'Michel Platini')]
[('player', 'http://www.wikidata.org/entity/Q57183'), ('name', 'Matthias Sammer')]
[('player', 'http://www.wikidata.org/entity/Q230046'), ('name', 'Gianni Rivera')]
[('player', 'http://www.wikidata.org/entity/Q220700'), ('name', 'Omar Sívori')]
[('player', 'http://www.wikidata.org/entity/Q43744'), ('name', 'Gerd Müller')]
[('player', 'http://www.wikidata.org/entity/Q123118'), ('name', 'Raymond Kopa')]
[('player', 'http://www.wikidata.org/entity/Q4457'), ('name', 'Franz Beckenbauer')]
[('player', 'http://www.wikidata.org/entity/Q102027'), ('name', 'Fabio Cannavaro')]
[('player', 'http://www.wikidata.org/entity/Q13494'), ('name', 'Jean-Pierre Papin')]
[('player', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus')]
[('player', 'http://www.wikidata.org/entity/Q1835'), ('name', 'Zinedine Zidane')]
[('player', 'http://www.wikidata.org/entity/Q483577'), ('name', 'Rivaldo')]
[('player', 

Final query for this task

In [54]:
#players who won Bd'Or with citizenship in a country which won at least 2 FWC
queryString = """
SELECT DISTINCT ?player ?name
WHERE {          
    ?player sc:name ?name;
            wdt:P106 wd:Q937857; #football players
            wdt:P27 ?country;    #citizenship
            wdt:P166 wd:Q166177. #won ballon d'or
    {
    SELECT DISTINCT ?country (COUNT(?comp) AS ?Nwin) WHERE {      
        ?comp wdt:P3450 wd:Q19317; #comp a FWC
              wdt:P1346 ?team.     #team is the winner
        ?team wdt:P17 ?country.    #team represents country
    }
    GROUP BY ?country ?name
    HAVING(COUNT(?comp) > 1)
    LIMIT 100
    }
   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q4261'), ('name', 'Michel Platini')]
[('player', 'http://www.wikidata.org/entity/Q57183'), ('name', 'Matthias Sammer')]
[('player', 'http://www.wikidata.org/entity/Q230046'), ('name', 'Gianni Rivera')]
[('player', 'http://www.wikidata.org/entity/Q220700'), ('name', 'Omar Sívori')]
[('player', 'http://www.wikidata.org/entity/Q43744'), ('name', 'Gerd Müller')]
[('player', 'http://www.wikidata.org/entity/Q123118'), ('name', 'Raymond Kopa')]
[('player', 'http://www.wikidata.org/entity/Q4457'), ('name', 'Franz Beckenbauer')]
[('player', 'http://www.wikidata.org/entity/Q102027'), ('name', 'Fabio Cannavaro')]
[('player', 'http://www.wikidata.org/entity/Q13494'), ('name', 'Jean-Pierre Papin')]
[('player', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus')]
[('player', 'http://www.wikidata.org/entity/Q1835'), ('name', 'Zinedine Zidane')]
[('player', 'http://www.wikidata.org/entity/Q483577'), ('name', 'Rivaldo')]
[('player', 